In [1]:
%load_ext autoreload
%autoreload 2

import os
import time
import numpy as np
from itertools import combinations

from pydrake.common import FindResourceOrThrow, RandomGenerator
from pydrake.geometry import Box, Role, SceneGraph
from pydrake.math import RigidTransform
from pydrake.multibody.parsing import LoadModelDirectives, Parser, ProcessModelDirectives
from pydrake.multibody.plant import CoulombFriction, AddMultibodyPlantSceneGraph, MultibodyPlant
from pydrake.multibody.tree import RevoluteJoint, SpatialInertia, UnitInertia
from pydrake.systems.analysis import Simulator
from pydrake.systems.framework import DiagramBuilder
from pydrake.systems.meshcat_visualizer import ConnectMeshcatVisualizer
from pydrake.systems.primitives import TrajectorySource
from pydrake.systems.rendering import MultibodyPositionToGeometryPose
from pydrake.trajectories import PiecewisePolynomial

from pydrake.planning.common_robotics_utilities import (
    MakeKinematicLinearRRTNearestNeighborsFunction,
    MakeKinematicLinearBiRRTNearestNeighborsFunction,
    MakeRRTTimeoutTerminationFunction,
    MakeBiRRTTimeoutTerminationFunction,
    PropagatedState,
    RRTPlanSinglePath,
    BiRRTPlanSinglePath,
    SimpleRRTPlannerState,
    Graph, GraphNode,
    GrowRoadMap,
    UpdateRoadMapEdges,
    QueryPath,
    LazyQueryPath,
    AddNodeToRoadmap,
    NNDistanceDirection)

from meshcat import Visualizer

In [2]:
# Setup meshcat
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
proc, zmq_url, web_url = start_zmq_server_as_subprocess(server_args=[])
vis = Visualizer(zmq_url=zmq_url)

# Sporadically need to run `pkill -f meshcat`

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7004/static/


# Visualize Model

In [3]:
model_file = "drake/planning/models/planar_iiwa_dense_collision_welded_gripper.yaml"
# model_file = "drake/planning/models/planar_iiwa_simple_collision_welded_gripper.yaml"

viz_role = Role.kIllustration
# viz_role = Role.kProximity

In [4]:
vis.delete()
display(vis.jupyter_cell())

builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.0)
parser = Parser(plant)
parser.package_map().Add( "wsg_50_description", os.path.dirname(FindResourceOrThrow(
            "drake/manipulation/models/wsg_50_description/package.xml")))

table_dim = np.array([4, 4, 0.2])
table = plant.AddRigidBody("table", SpatialInertia(
        mass=1.0, p_PScm_E=np.array([0., 0., 0.]), G_SP_E=UnitInertia(1.0, 1.0, 1.0)))
plant.WeldFrames(plant.world_frame(), table.body_frame(), RigidTransform(p=np.array([0, 0, -table_dim[2]/2])))
plant.RegisterVisualGeometry(table, RigidTransform(), Box(*table_dim), "table_vis",
                             np.array([0.5, 0.5, 0.5, 1.]))
plant.RegisterCollisionGeometry(table, RigidTransform(), Box(*table_dim), "table_collision",
                                CoulombFriction(0.9, 0.8))

directives_file = FindResourceOrThrow(model_file)
directives = LoadModelDirectives(directives_file)
models = ProcessModelDirectives(directives, plant, parser)
[iiwa, wsg, shelves] = models

plant.Finalize()

visualizer = ConnectMeshcatVisualizer(builder, scene_graph, zmq_url=zmq_url,
                                      delete_prefix_on_load=False, role=viz_role)
diagram = builder.Build()

q0 = [-0.2, -1.2, 1.6]
index = 0
for joint_index in plant.GetJointIndices(iiwa.model_instance):
    joint = plant.get_mutable_joint(joint_index)
    if isinstance(joint, RevoluteJoint):
        joint.set_default_angle(q0[index])
        index += 1

visualizer.load()
context = diagram.CreateDefaultContext()
plant_context = plant.GetMyContextFromRoot(context)
sg_context = scene_graph.GetMyContextFromRoot(context)
diagram.Publish(context)

Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6001...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7004/static/
Connected to meshcat-server.


# Visualize Trajectory

In [5]:
def visualize_trajectory(traj):
    builder = DiagramBuilder()
    
    scene_graph = builder.AddSystem(SceneGraph())
    plant = MultibodyPlant(time_step=0.0)
    plant.RegisterAsSourceForSceneGraph(scene_graph)
    parser = Parser(plant)
    parser.package_map().Add( "wsg_50_description", os.path.dirname(FindResourceOrThrow(
                "drake/manipulation/models/wsg_50_description/package.xml")))

    table_dim = np.array([4, 4, 0.2])
    table = plant.AddRigidBody("table", SpatialInertia(
            mass=1.0, p_PScm_E=np.array([0., 0., 0.]), G_SP_E=UnitInertia(1.0, 1.0, 1.0)))
    plant.WeldFrames(plant.world_frame(), table.body_frame(), RigidTransform(p=np.array([0, 0, -table_dim[2]/2])))
    plant.RegisterVisualGeometry(table, RigidTransform(), Box(*table_dim), "table_vis",
                                 np.array([0.5, 0.5, 0.5, 1.]))
    plant.RegisterCollisionGeometry(table, RigidTransform(), Box(*table_dim), "table_collision",
                                    CoulombFriction(0.9, 0.8))
    
    directives_file = FindResourceOrThrow(model_file)
    directives = LoadModelDirectives(directives_file)
    models = ProcessModelDirectives(directives, plant, parser)
    
    plant.Finalize()

    to_pose = builder.AddSystem(MultibodyPositionToGeometryPose(plant))
    builder.Connect(to_pose.get_output_port(), scene_graph.get_source_pose_port(plant.get_source_id()))

    traj_system = builder.AddSystem(TrajectorySource(traj))
    builder.Connect(traj_system.get_output_port(), to_pose.get_input_port())
    
    meshcat = ConnectMeshcatVisualizer(builder, scene_graph, zmq_url=zmq_url,
                                       delete_prefix_on_load=False, role=viz_role)

    vis_diagram = builder.Build()
    simulator = Simulator(vis_diagram)
    meshcat.start_recording()
    simulator.AdvanceTo(traj.end_time())
    meshcat.publish_recording()

q_test = [0.4, 1.8, -0.5]
traj = PiecewisePolynomial.FirstOrderHold([0, 3], np.array([[-0.2, -1.2, 1.6], q_test]).T)
vis.delete()
display(vis.jupyter_cell())
visualize_trajectory(traj)

Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6001...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7004/static/
Connected to meshcat-server.


# Functions and Parameters for Planning 

In [7]:
q_start = np.array([0.5, -1.2, 0])
q_end = np.array([0.8, -1.6, -0.9])

goal_bias = 0.05
step_size = np.pi/16
collision_step_size = step_size/4
solve_timeout = 100

plant_context_planning = plant.GetMyContextFromRoot(context)

PositionUpperLimits = plant.GetPositionUpperLimits()
PositionLowerLimits = plant.GetPositionLowerLimits()

In [8]:
def sampling_cspace():
    if np.random.rand() < goal_bias:
        return q_end 
    return np.random.rand(3)*(PositionUpperLimits-PositionLowerLimits) + PositionLowerLimits

def check_goal_fn(q):
    return np.linalg.norm(q_end - q) < 1e-6

def distance_fn(q_1, q_2):
    return np.linalg.norm(q_2 - q_1)


def check_state_validity_fn(point):
    #check for collisions
    plant.SetPositions(plant_context_planning, point)
    query_object = plant.get_geometry_query_input_port().Eval(plant_context_planning)
    
    return not query_object.HasCollisions()

        
        
def check_edge_validity_fn(start, end):
    #use first order interpolation to check collisions along a path between start and end
    def checkEdgeCollisionFree(start, end, stepsize):
        num_steps = np.ceil(distance_fn(start, end)/stepsize)

        for step in range(int(num_steps)+1):
            interpolation_ratio = step / num_steps
            interpolated_point = start + interpolation_ratio*(end-start)
            
            if not check_state_validity_fn(interpolated_point):
                return False

        return True

    return checkEdgeCollisionFree(start, end, collision_step_size)

def extend_fn(nearest, sample, is_start_tree = None):
    extend = None
    extend_dist = distance_fn(nearest, sample)
    
    if extend_dist <= step_size:
        extend = sample
    else:
        extend = nearest + step_size/extend_dist * (sample - nearest)
    
    if not check_edge_validity_fn(nearest, extend):
        return []
    
    return [PropagatedState(state=extend, relative_parent_index=-1)]
    


# Use RRT to Plan Trajectory

In [8]:
rrt_tree = [SimpleRRTPlannerState(q_start)]


nearest_neighbor_fn = MakeKinematicLinearRRTNearestNeighborsFunction(distance_fn=distance_fn, use_parallel = False)

termination_fn = MakeRRTTimeoutTerminationFunction(solve_timeout)

single_result = RRTPlanSinglePath(
    tree=rrt_tree, sampling_fn=sampling_cspace,
    nearest_neighbor_fn =nearest_neighbor_fn,
    forward_propagation_fn=extend_fn,
    state_added_callback_fn=None,
    check_goal_reached_fn=check_goal_fn, goal_reached_callback_fn=None,
    termination_check_fn=termination_fn)

path = single_result.Path()

print(path)

if len(path) > 0:
    rrt_traj = PiecewisePolynomial.FirstOrderHold(list(range(len(path))), np.stack(path).T) 

    vis.delete()
    display(vis.jupyter_cell())
    visualize_trajectory(rrt_traj)

[]


# Use Bi-RRT to Plan Trajectory

In [9]:
# Reuse single RRT functions, but need to add connect
seed = 0
step_size = np.pi/32
collision_step_size = step_size/128

def connect_fn(nearest, sample, is_start_tree):
    total_dist = distance_fn(nearest, sample)
    total_steps = int(np.ceil(total_dist / step_size))        
        
    propagated_states = []
    parent_offset = -1
    current = nearest
    for steps in range(total_steps):
        current_target = None
        target_dist = distance_fn(current, sample)
        if target_dist > step_size:
            #interpolate
            current_target = current + step_size/target_dist * (sample - current)
            
        elif target_dist < 1e-6:
            break
        else:
            current_target = sample
        
        if not check_edge_validity_fn(current, current_target):
            return propagated_states
    
                    
        propagated_states.append(PropagatedState(state=current_target, relative_parent_index=parent_offset))
        parent_offset += 1
        current = current_target

    return propagated_states
        
        
def states_connected_fn(source, target, is_start_tree):
    return np.linalg.norm(source - target) < 1e-6
        
start_tree = [SimpleRRTPlannerState(q_start)]
end_tree = [SimpleRRTPlannerState(q_end)]


nearest_neighbor_fn = MakeKinematicLinearBiRRTNearestNeighborsFunction(distance_fn=distance_fn, use_parallel = False)

termination_fn = MakeBiRRTTimeoutTerminationFunction(solve_timeout)

extend_result = BiRRTPlanSinglePath(
    start_tree=start_tree, goal_tree=end_tree,
    state_sampling_fn=sampling_cspace,
    nearest_neighbor_fn=nearest_neighbor_fn, propagation_fn=extend_fn,
    state_added_callback_fn=None,
    states_connected_fn=states_connected_fn,
    goal_bridge_callback_fn=None,
    tree_sampling_bias=0.5, p_switch_tree=0.25,
    termination_check_fn=termination_fn, rng=RandomGenerator(seed))

connect_result = BiRRTPlanSinglePath(
    start_tree=start_tree, goal_tree=end_tree,
    state_sampling_fn=sampling_cspace,
    nearest_neighbor_fn=nearest_neighbor_fn, propagation_fn=connect_fn,
    state_added_callback_fn=None,
    states_connected_fn=states_connected_fn,
    goal_bridge_callback_fn=None,
    tree_sampling_bias=0.5, p_switch_tree=0.25,
    termination_check_fn=termination_fn, rng=RandomGenerator(seed))

extend_path = extend_result.Path()
connect_path = connect_result.Path()

<ipython-input-8-e5e286c59a58>:28: RuntimeWarning: invalid value encountered in double_scalars
  interpolation_ratio = step / num_steps


In [10]:
birrt_traj = PiecewisePolynomial.FirstOrderHold(list(range(len(connect_path))), np.stack(connect_path).T) 
#birrt_traj = PiecewisePolynomial.FirstOrderHold(list(range(len(extend_path))), np.stack(extend_path).T) 

vis.delete()
display(vis.jupyter_cell())
visualize_trajectory(birrt_traj)

Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6001...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7002/static/
Connected to meshcat-server.


## Use PRM to Plan Trajectory

In [130]:
def prm_sampling_fn():
    return np.random.rand(3)*(PositionUpperLimits-PositionLowerLimits) + PositionLowerLimits

def roadmap_termination_fn(current_roadmap_size):
    return current_roadmap_size >= roadmap_size

### Sample Uniformly at first

In [140]:
start = time.time()
# Sample uniformly
K = 5
roadmap_size = 250 # increase this number for more inital coverage 

#roadmap = Graph([GraphNode(q_start), GraphNode(q_end)])
roadmap = Graph()

GrowRoadMap(roadmap, prm_sampling_fn, distance_fn,
        check_state_validity_fn, check_edge_validity_fn,
        roadmap_termination_fn, K, False, True, False)

#UpdateRoadMapEdges(roadmap, check_edge_validity_fn, distance_fn, False)
print(f"Time: {np.ceil(time.time() - start)}s")

Time: 25.0s


### Identify sparse Milestones and sample around it to increase density

In [141]:
start = time.time()
#identify milestones with only a few connections and sample around it
max_iter = 100
cnt = 0

while cnt < max_iter:
    milestones = roadmap.GetNodesImmutable()
    sparse_milestones = list(filter(lambda node: len(node.GetInEdgesImmutable()) < K , milestones))
    print(f"Sparse Count: {len(sparse_milestones)}")
    if len(sparse_milestones) == 0:
        #not sparse anymore
        break
    
    roadmap_size += 50
    
    sparse_qs = np.stack([node.GetValueImmutable()for node in sparse_milestones])
    
    lower_bound = np.maximum(sparse_qs - step_size, PositionLowerLimits)
    upper_bound = np.minimum(sparse_qs + step_size, PositionUpperLimits)

    def sparse_sampling_fn():
        q_idx = np.random.randint(lower_bound.shape[0])
        return np.random.rand(3)*(upper_bound[q_idx, :] -lower_bound[q_idx, :]) + lower_bound[q_idx, :]
    
    GrowRoadMap(roadmap, sparse_sampling_fn, distance_fn,
        check_state_validity_fn, check_edge_validity_fn,
        roadmap_termination_fn, K, False, True, False)

    cnt +=1
    
print(f"Time: {np.ceil(time.time() - start)}s")

Sparse Count: 15
Sparse Count: 20
Sparse Count: 4
Sparse Count: 1
Sparse Count: 0
Time: 4.0s


In [142]:
print(f"Increased roadmap to {roadmap_size}")

Increased roadmap to 450


### Altought all nodes have at least K neighbors, there is a chance that our graph is disconnected

In [143]:
def return_subgraphs_idx(roadmap):
    milestones_idx = set(range(len(roadmap.GetNodesMutable())))
    subgraphs = []
    
    while len(milestones_idx) != 0:
        #pick a random milestone
        subgraph = set()
        root = milestones_idx.pop()
        queue = [root]
    
        #start expanding bfs style 
        while len(queue) != 0:
            expand_node_idx = queue.pop(0)
        
            if expand_node_idx in subgraph:
                #already expanded
                continue
            
            subgraph.add(expand_node_idx)
            expand_node = roadmap.GetNodeImmutable(expand_node_idx)
            for child in [edge.GetToIndex() for edge in expand_node.GetOutEdgesImmutable()]:
                queue.append(child)
    
        subgraphs.append(subgraph)
        #remove subgraph from milestones 
        milestones_idx -= set(subgraph)
    return milestones_idx, subgraphs

milestones_idx, subgraphs = return_subgraphs_idx(roadmap)

print(f"Found {len(subgraphs)} subgraphs, with sizes {list(map(len,subgraphs))}")

Found 5 subgraphs, with sizes [330, 9, 6, 10, 95]


## Compute center of subgraphs for implosions

In [144]:
center_nodes = []
for graph in subgraphs:
    graph_qs = []
    for q_idx in graph:
        graph_qs.append(roadmap.GetNodeImmutable(q_idx).GetValueImmutable())
    graph_qs = np.stack(graph_qs)
    mean_q = graph_qs.mean(axis=0)
    center_nodes.append(mean_q)

In [145]:
center_nodes

[array([ 0.01568963, -0.10807747,  0.01112872]),
 array([ 1.9151396 ,  0.76834599, -2.03627725]),
 array([1.60390609, 1.37885795, 1.49537835]),
 array([ 0.44069248, -1.748413  , -1.30914936]),
 array([ 0.61725459, -1.1132594 , -0.56739574])]

In [137]:
max_dist = np.linalg.norm(np.max(center_nodes, axis = 0) - np.min(center_nodes, axis = 0))

In [146]:
start = time.time()
def sample_spherical(offset, r=1, ndim=3):
    vec = np.random.randn(3)
    vec /= np.linalg.norm(vec)
    vec *=r #resize
    vec += offset
    return vec

gamma = 0.7

for q_center in center_nodes:
    r = max_dist #inital implosion radius 
    
    implosion_steps = int(abs(np.ceil(np.log(r/step_size)/np.log(gamma))))
    
    def implosion_sampling_fn(r=r):
        q_rand = sample_spherical(q_center, r)
        while True:
            if all(q_rand >= PositionLowerLimits) and all(q_rand <= PositionUpperLimits):
                return q_rand
            r *= gamma #decay
            q_rand = sample_spherical(q_center, r)
            
    for _ in range(implosion_steps):
        #perimeter of sphere
        roadmap_size += int(np.ceil(2*np.pi*r/step_size)) #keep constant density along implostion
        GrowRoadMap(roadmap, implosion_sampling_fn, distance_fn,
            check_state_validity_fn, check_edge_validity_fn,
            roadmap_termination_fn, K, False, True, False)
        r *= gamma #decay
print(f"Time: {np.ceil(time.time() - start)}s")

Time: 127.0s


In [147]:
milestones_idx, subgraphs = return_subgraphs_idx(roadmap)

print(f"Found {len(subgraphs)} subgraphs, with sizes {list(map(len,subgraphs))}")
print(f"Increased roadmap to {roadmap_size}")

Found 8 subgraphs, with sizes [3354, 24, 2, 6, 10, 2, 1, 1]
Increased roadmap to 3400


### Now try to connect subgraphs

In [148]:
start = time.time()
def RRT_Connect(connect_pairs):
    for q_ini_idx, q_final_idx in connect_pairs:
        q_ini = roadmap.GetNodeImmutable(q_ini_idx).GetValueImmutable()
        q_final = roadmap.GetNodeImmutable(q_final_idx).GetValueImmutable()
    
        start_tree = [SimpleRRTPlannerState(q_ini)]
        end_tree = [SimpleRRTPlannerState(q_final)]


        nearest_neighbor_fn = MakeKinematicLinearBiRRTNearestNeighborsFunction(distance_fn=distance_fn, use_parallel = False)

        termination_fn = MakeBiRRTTimeoutTerminationFunction(solve_timeout)
    
        connect_result = BiRRTPlanSinglePath(
            start_tree=start_tree, goal_tree=end_tree,
            state_sampling_fn=sampling_cspace,
            nearest_neighbor_fn=nearest_neighbor_fn, propagation_fn=connect_fn,
            state_added_callback_fn=None,
            states_connected_fn=states_connected_fn,
            goal_bridge_callback_fn=None,
            tree_sampling_bias=0.5, p_switch_tree=0.25,
            termination_check_fn=termination_fn, rng=RandomGenerator(seed))
        
        #add all nodes along the path to roadmap
        for q in connect_result.Path():
            AddNodeToRoadmap(q, NNDistanceDirection(), roadmap, distance_fn,check_edge_validity_fn, K, False,True,False )
            
mothernodes = list(map(lambda graph: np.random.choice(list(graph)), subgraphs))
#shuffle nodes
mothernodes = sorted(mothernodes, key=lambda x: np.random.randint(len(mothernodes)))

#try to connect a random chain 
chain = zip(mothernodes[:-1], mothernodes[1:])
RRT_Connect(chain)

_, subgraphs = return_subgraphs_idx(roadmap)

if len(subgraphs) > 0:
    print("Try to fully connect subgraphs")
    mothernodes = list(map(lambda graph: np.random.choice(list(graph)), subgraphs))
    RRT_Connect(combinations(mothernodes,2))
    

print(f"Found {len(subgraphs)} subgraphs, with sizes {list(map(len,subgraphs))}")
print(f"Time: {np.ceil(time.time() - start)}s")

Try to fully connect subgraphs


KeyboardInterrupt: 

### Check for connectivity

In [149]:
milestones_idx, subgraphs = return_subgraphs_idx(roadmap)

print(f"Found {len(subgraphs)} subgraphs, with sizes {list(map(len,subgraphs))}")

Found 5 subgraphs, with sizes [3708, 24, 2, 2, 1]


## Lets Plan!

In [151]:
start = time.time()
prm_path = QueryPath([q_start], [q_end], roadmap, distance_fn,
                check_edge_validity_fn, K,
                use_parallel=False,
                distance_is_symmetric=True,
                add_duplicate_states=False,
                limit_astar_pqueue_duplicates=True).Path()
print(f"Time: {(time.time() - start)}s")

Time: 0.06011652946472168s


## Lets Benchmark

In [159]:
N = 400
failed = 0
times = []
for i in range(N):
    start_time =  time.time()
    goal = prm_sampling_fn()
    start = prm_sampling_fn()
    prm_path = QueryPath([start], [goal], roadmap, distance_fn,
                check_edge_validity_fn, K,
                use_parallel=False,
                distance_is_symmetric=True,
                add_duplicate_states=False,
                limit_astar_pqueue_duplicates=True).Path()
    
    if len(prm_path) == 0:
        failed +=1
    
    delta = time.time() - start_time
    times.append(delta)
    
print(f"Number of samples: {N}")
print(f"Avg solve time: {np.round(sum(times)/N,3)} s")
print(f"Min solve time: {np.round(min(times))} s")
print(f"Max solve time: {np.round(min(times))} s")
print(f"Failed paths: {failed/N*100} %")

    

Number of samples: 400
Avg solve time: 0.095 s
Min solve time: 0.0 s
Max solve time: 0.0 s
Failed paths: 56.49999999999999 %


## Benchmark with Backup plan

In [160]:
N = 40
failed = 0
times = []
reconnects = 0
for i in range(N):
    start_time =  time.time()
    goal = prm_sampling_fn()
    start = prm_sampling_fn()
    prm_path = QueryPath([start], [goal], roadmap, distance_fn,
                check_edge_validity_fn, K,
                use_parallel=False,
                distance_is_symmetric=True,
                add_duplicate_states=False,
                limit_astar_pqueue_duplicates=True).Path()
    
    if len(prm_path) == 0:
        #attempt to reconnect
        reconnects +=1
        start_tree = [SimpleRRTPlannerState(start)]
        end_tree = [SimpleRRTPlannerState(goal)]

        nearest_neighbor_fn = MakeKinematicLinearBiRRTNearestNeighborsFunction(distance_fn=distance_fn, use_parallel = False)

        termination_fn = MakeBiRRTTimeoutTerminationFunction(solve_timeout)
    
        connect_result = BiRRTPlanSinglePath(
            start_tree=start_tree, goal_tree=end_tree,
            state_sampling_fn=sampling_cspace,
            nearest_neighbor_fn=nearest_neighbor_fn, propagation_fn=connect_fn,
            state_added_callback_fn=None,
            states_connected_fn=states_connected_fn,
            goal_bridge_callback_fn=None,
            tree_sampling_bias=0.5, p_switch_tree=0.25,
            termination_check_fn=termination_fn, rng=RandomGenerator(seed))
        
        if len(connect_result.Path()) == 0:
            failed +=1
        #add all nodes along the path to roadmap
        for q in connect_result.Path():
            AddNodeToRoadmap(q, NNDistanceDirection(), roadmap, distance_fn,check_edge_validity_fn, K, False,True,False )
    
    delta = time.time() - start_time
    times.append(delta)
    
print(f"Number of samples: {N}")
print(f"Avg solve time: {np.round(sum(times)/N,3)} s")
print(f"Min solve time: {np.round(min(times))} s")
print(f"Max solve time: {np.round(min(times))} s")
print(f"Failed paths: {failed/N*100} %")
print(f"Reconnected paths: {reconnects/N*100} %")

Number of samples: 40
Avg solve time: 45.101 s
Min solve time: 0.0 s
Max solve time: 0.0 s
Failed paths: 45.0 %
Reconnected paths: 45.0 %


In [20]:
prm_traj = PiecewisePolynomial.FirstOrderHold(list(range(len(prm_path))), np.stack(prm_path).T) 

vis.delete()
display(vis.jupyter_cell())
visualize_trajectory(prm_traj)

Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6001...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7002/static/
Connected to meshcat-server.
